In [ ]:
import os
import glob
import re
import itertools
import time

import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_rows = None
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import parsl
from parsl.config import Config
from parsl.executors import HighThroughputExecutor

config = Config(
    executors=[
        HighThroughputExecutor(max_workers=40)
    ],
    strategy=None
)
parsl.load(config)

from src.utils import get_consensus

In [ ]:
benchmarking_dir = '/cephfs/users/annawoodard/gene-fusion/FusionBenchmarking'
analysis_tag = 'sim_50'
analysis_dir = 'simulated_data/{}'.format(analysis_tag)
ctat_dir = '/cephfs/users/annawoodard/gene-fusion/GRCh38_gencode_v31_CTAT_lib_Oct012019.plug-n-play/ctat_genome_lib_build_dir'

In [ ]:
command = (
    "docker run -v {}:/data trinityctat/fusionbenchmarking "
    "bash -c "
    "'find /data/{}/samples -type f | /data/util/make_file_listing_input_table.pl'"
).format(benchmarking_dir, analysis_dir)

!$command > $benchmarking_dir/$analysis_dir/simulated_datafusion_result_file_listing.dat

In [ ]:
command = (
    "docker run -v {}:/data trinityctat/fusionbenchmarking "
    "bash -c "
    "'/usr/local/src/FusionBenchmarking/benchmarking/collect_preds.pl "
    "/data/{}/simulated_datafusion_result_file_listing.dat'"
).format(benchmarking_dir, analysis_dir)
!$command > $benchmarking_dir/$analysis_dir/preds.collected

In [ ]:
command = (
    "docker run -v {}:/data trinityctat/fusionbenchmarking "
    "bash -c '"
    "/usr/local/src/FusionBenchmarking/benchmarking/map_gene_symbols_to_gencode.pl "
    "/data/{}/preds.collected "
    "/data/resources/genes.coords.gz /data/resources/genes.aliases'"
).format(benchmarking_dir, analysis_dir)

!$command > $benchmarking_dir/$analysis_dir/preds.collected.gencode_mapped

In [ ]:
!docker run -v /cephfs/users/annawoodard/gene-fusion/GRCh38_gencode_v31_CTAT_lib_Oct012019.plug-n-play/ctat_genome_lib_build_dir:/ctat_genome_lib_build_dir -v /cephfs/users/annawoodard/gene-fusion/data/interim/fusions.tsv:/fusions.tsv trinityctat/starfusion:1.8.0 /usr/local/src/STAR-Fusion/FusionInspector/FusionAnnotator/FusionAnnotator --annotate /fusions.tsv --full --genome_lib_dir /ctat_genome_lib_build_dir/ > /cephfs/users/annawoodard/gene-fusion/data/interim/annotated_fusions.tsv

In [ ]:
command = (
    "docker run "
    "-v {}:/ctat_genome_lib_build_dir "
    "-v {}:/data "
    "trinityctat/fusionbenchmarking "
    "bash -c '"
    "FusionAnnotator/FusionAnnotator "
    "--annotate /data/{}/preds.collected.gencode_mapped -C 2 --genome_lib_dir /ctat_genome_lib_build_dir/'"
).format(ctat_dir, benchmarking_dir, analysis_dir)

!$command > $benchmarking_dir/$analysis_dir/preds.collected.gencode_mapped.wAnnot

In [ ]:
command = (
    "docker run -v {}:/data trinityctat/fusionbenchmarking "
    "bash -c "
    "'/usr/local/src/FusionBenchmarking/benchmarking/filter_collected_preds.pl "
    "/data/{}/preds.collected.gencode_mapped.wAnnot'"
).format(benchmarking_dir, analysis_dir)
!$command > $benchmarking_dir/$analysis_dir/preds.collected.gencode_mapped.wAnnot.filt

In [ ]:
command = (
    "docker run -v {}/..:/data trinityctat/fusionbenchmarking "
    "bash -c "
    "'chmod -R a+w /data/FusionBenchmarking/{}'"
).format(benchmarking_dir, analysis_dir)
!$command

In [ ]:
predictions = pd.read_csv(
    os.path.join(benchmarking_dir, analysis_dir, 'preds.collected.gencode_mapped.wAnnot.filt'),
    sep='\t',
    skiprows=1,
    header=None,
    names=['sample', 'prog', 'fusion', 'J', 'S', 'mapped_gencode_A_gene_list', 'mapped_gencode_B_gene_list', 'annotations']
)

In [ ]:
predictions['gene_a'] = [sorted(i.split(','))[0] for i in predictions.mapped_gencode_A_gene_list]
predictions['gene_b'] = [sorted(i.split(','))[0] for i in predictions.mapped_gencode_B_gene_list]
predictions['normalized_fusion'] = predictions[['gene_a', 'gene_b']].apply(lambda x: '--'.join(sorted(x)), axis=1)
predictions['sum_J_S'] = predictions.J + predictions.S

In [ ]:
samples = sorted(predictions['sample'].unique())
fusion_sets = pd.DataFrame(data={
    'prog': [
        prog 
        for prog in sorted(predictions.prog.unique())
        for sample in samples
    ],
    'sample': [
        sample
        for prog in sorted(predictions.prog.unique())
        for sample in samples
    ],
    'fusion_set': [
        set(
            predictions[
                (predictions['sample'] == sample) & 
                (predictions['prog'] == prog)
            ]['normalized_fusion'].unique()
        ) 
        for prog in sorted(predictions.prog.unique())
        for sample in samples
    ]
}
)

In [ ]:
auc = pd.read_csv(
    os.path.join(benchmarking_dir, analysis_dir, '__analyze_strict/all.AUC.dat'),
    sep='\t',
    header=None,
    names=['prog', 'auc']
)

progs = auc.prog.unique()
mean_auc = pd.DataFrame(data={
    'prog': progs,
    'mean_auc': [auc[auc.prog == prog].mean()[0] for prog in progs]
    }
)

In [ ]:
mean_auc

In [ ]:
start = time.time()

top_prog = mean_auc.sort_values(by='mean_auc', ascending=False)['prog'][0]

futures = []
for num_progs in range(2, 10):
    top_progs = mean_auc.sort_values(by='mean_auc', ascending=False)['prog'][:num_progs]
    for quorum in range(2, num_progs + 1):
        futures += [
            (
                get_consensus(
                    fusion_sets[
                        (fusion_sets['sample'] == sample) & 
                        (fusion_sets['prog'].str.contains('|'.join(top_progs)))
                    ].fusion_set.to_list(),
                    quorum
                ),
                sample,
                top_progs,
                quorum
            )
            for sample in samples
        ]

ensemble_predictions = []
for f, sample, top_progs, quorum in futures:
    ensemble_fusions = list(f.result())
    
    indices = [
        predictions[
            (predictions['sample'] == sample) &
            (predictions['normalized_fusion'] == ensemble_fusions[i]) &
            (predictions['prog'].str.contains('|'.join(top_progs)))
        ].sum_J_S.idxmax()
        for i in range(len(ensemble_fusions))
    ]
    ensemble_predictions.append(predictions.loc[indices])
    prog = 'quorum_{}(top_{})'.format(quorum, len(top_progs))
    ensemble_predictions[-1].prog = prog
    print('finished {} ({} fusions for {}) in {:.0f} seconds'.format(prog, len(indices), sample, time.time() - start))
    
    union_ensemble_fusions = list(set.union(
        fusion_sets[
            (fusion_sets['sample'] == sample) & 
            (fusion_sets['prog'] == top_prog)
        ].fusion_set.to_list()[0],
        ensemble_fusions
    ))
    
    indices = [
        predictions[
            (predictions['sample'] == sample) &
            (predictions['normalized_fusion'] == union_ensemble_fusions[i]) &
            (predictions['prog'].str.contains('|'.join(top_progs)))
        ].sum_J_S.idxmax()
        for i in range(len(union_ensemble_fusions))
    ]
    ensemble_predictions.append(predictions.loc[indices])
    prog = 'union_{}_quorum_{}(top_{})'.format(top_prog, quorum, len(top_progs))
    ensemble_predictions[-1].prog = prog
    
    print('finished {} ({} fusions) in {:.0f} seconds'.format(prog, len(indices), time.time() - start))

print('finished in {:.0f} seconds'.format(time.time() - start))

In [ ]:
predictions = pd.concat([predictions] + ensemble_predictions)

In [ ]:
predictions.to_csv(
    os.path.join(benchmarking_dir, analysis_dir, 'preds.collected.gencode_mapped.wAnnot.filt'),
    sep='\t',
    index=False,
    columns=['sample', 'prog', 'fusion', 'J', 'S', 'mapped_gencode_A_gene_list', 'mapped_gencode_B_gene_list', 'annotations']
)

In [ ]:
command = (
    "docker run -v {}/..:/data trinityctat/fusionbenchmarking "
    "bash -c "
    "'rm -rf /data/FusionBenchmarking/{}/__*'"
).format(benchmarking_dir, analysis_dir)
!$command

In [ ]:
command = (
    "docker run -v {}/..:/data trinityctat/fusionbenchmarking "
    "bash -c "
    "'cd /data/FusionBenchmarking/{}; "
    "/data/FusionBenchmarking/simulated_data/analyze_simulated_data.pl "
    "{}.truth_set.dat {}.fusion_TPM_values.dat'"
).format(benchmarking_dir, analysis_dir, analysis_tag, analysis_tag)
!$command

In [ ]:
auc = pd.read_csv(
    os.path.join(benchmarking_dir, analysis_dir, '__analyze_strict/all.AUC.dat'),
    sep='\t',
    header=None,
    names=['prog', 'auc']
)

progs = auc.prog.unique()
mean_auc = pd.DataFrame(data={
    'prog': progs,
    'mean_auc': [auc[auc.prog == prog].mean()[0] for prog in progs]
    }
)

In [ ]:
ax = sns.boxplot(x="prog", y="auc", data=auc)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

In [ ]:
pd.options.display.max_colwidth = 500
mean_auc.sort_values(by='mean_auc', ascending=False)

In [ ]:
(.8-.76)/.76*100

In [ ]:
set.intersection({1, 2}, {2, 3})

In [3]:
import itertools
sets = ['A', 'B', 'C', 'D']
result = []
for i in range(1, len(sets) + 1):
    result += list(itertools.combinations(sets, i))

print(len(result), result)
                

15 [('A',), ('B',), ('C',), ('D',), ('A', 'B'), ('A', 'C'), ('A', 'D'), ('B', 'C'), ('B', 'D'), ('C', 'D'), ('A', 'B', 'C'), ('A', 'B', 'D'), ('A', 'C', 'D'), ('B', 'C', 'D'), ('A', 'B', 'C', 'D')]


In [5]:
combos = []
for i in range(1, len(result) + 1):
    combos += list(itertools.combinations(result, i))
    print(len(combos))
print(len(combos), combos)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

